# Estimation of damage caused to housing in Miami due to hurricanes

So, this notebook is going to be used to study the effect of a hurricane on the city of Miami.
Yes, the scale is smaller. Yes, we went from Florida to only Miami, but that's the only dataset we have right now!

Let us load the dataset and visualize it before moving forward.

In [1]:
import duckdb
import pandas as pd
import folium

In [23]:
center_lat, center_lon = 25.8910306063, -80.1605605249
radius_km = 2  # 2 km

In [24]:
df = duckdb.execute("SELECT * FROM '../data/miami-housing.csv'").df()

In [25]:
df.head()

,LATITUDE,LONGITUDE,PARCELNO,SALE_PRC,LND_SQFOOT,TOT_LVG_AREA,SPEC_FEAT_VAL,RAIL_DIST,OCEAN_DIST,WATER_DIST,CNTR_DIST,SUBCNTR_DI,HWY_DIST,age,avno60plus,month_sold,structure_quality
0,25.891031,-80.160561,622280070620,440000.0,9375,1753,0,2815.9,12811.4,347.6,42815.3,37742.2,15954.9,67,0,8,4
1,25.891324,-80.153968,622280100460,349000.0,9375,1715,0,4359.1,10648.4,337.8,43504.9,37340.5,18125.0,63,0,9,4
2,25.891334,-80.153740,622280100470,800000.0,9375,2276,49206,4412.9,10574.1,297.1,43530.4,37328.7,18200.5,61,0,2,4
3,25.891765,-80.152657,622280100530,988000.0,12450,2058,10033,4585.0,10156.5,0.0,43797.5,37423.2,18514.4,63,0,9,4
4,25.891825,-80.154639,622280100200,755000.0,12800,1684,16681,4063.4,10836.8,326.6,43599.7,37550.8,17903.4,42,0,7,4


In [26]:
import math

# Haversine formula to calculate great-circle distance
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of Earth in kilometers. Use 3959 for miles.
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)
    
    a = math.sin(delta_phi / 2) ** 2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    return R * c

# Filter ZIP codes within the radius
df['distance'] = df.apply(
    lambda row: haversine(center_lat, center_lon, row['LATITUDE'], row['LONGITUDE']), axis=1
)
zipcodes_in_radius = df[df['distance'] <= radius_km]

In [27]:
m = folium.Map(location=[center_lat, center_lon], zoom_start=10)

# Add markers for each ZIP code
for _, row in zipcodes_in_radius.iterrows():
    folium.Marker(
        location=[row['LATITUDE'], row['LONGITUDE']],
        popup=f"ZIP: {row['SALE_PRC']}, Distance: {row['distance']} km",
        icon=folium.Icon(color="blue", icon="info-sign")
    ).add_to(m)

m